<a href="https://colab.research.google.com/github/faizanali02/googlecolab/blob/main/233574_ai_lab07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ================================================================
#                AI CSP: UNIVERSITY TIMETABLING SYSTEM
#          MRV + Forward Checking + Colored HTML GUI Table
# ================================================================

import pandas as pd
import itertools
from IPython.display import HTML, display

# ------------------------------------------------
# 1) AUTO-GENERATE SAMPLE DATASETS
# ------------------------------------------------

courses = pd.DataFrame({
    "Course": ["AI", "DS", "OOP", "DB", "SE"],
    "Teacher": ["Sir A", "Sir B", "Sir C", "Sir A", "Sir D"],
    "Subject": ["AI", "Data Science", "Programming", "Databases", "Software Eng"]
})

rooms = pd.DataFrame({
    "Room": ["R1", "R2", "R3"],
    "Capacity": [40, 30, 50]
})

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
times = ["9-10", "10-11", "11-12", "1-2", "2-3"]

# ------------------------------------------------
# 2) BUILD THE CSP VARIABLES & DOMAINS
# ------------------------------------------------

variables = list(courses["Course"])

domain = list(itertools.product(rooms["Room"], days, times))
domains = {course: domain.copy() for course in variables}

# ------------------------------------------------
# 3) CONSTRAINT CHECKING FUNCTION
# ------------------------------------------------

def is_consistent(course, assignment, value):
    room, day, time = value

    for c, v in assignment.items():
        r2, d2, t2 = v

        if c == course:
            continue

        # Constraint 1: No course can share the same room at same time
        if (day == d2) and (time == t2) and (room == r2):
            return False

        # Constraint 2: Same teacher cannot take two classes at same time
        t1 = courses[courses["Course"] == course]["Teacher"].values[0]
        t2_ = courses[courses["Course"] == c]["Teacher"].values[0]

        if t1 == t2_ and (day == d2) and (time == t2):
            return False

    return True

# ------------------------------------------------
# 4) MRV: Minimum Remaining Value heuristic
# ------------------------------------------------

def select_unassigned_var(assign, domains):
    unassigned = [v for v in variables if v not in assign]
    return min(unassigned, key=lambda var: len(domains[var]))

# ------------------------------------------------
# 5) FORWARD CHECKING
# ------------------------------------------------

def forward_check(course, value, domains, assignment):
    new_domains = {c: domains[c].copy() for c in domains}

    for other in variables:
        if other != course and other not in assignment:
            for v in domains[other]:
                if not is_consistent(other, {**assignment, course: value}, v):
                    if v in new_domains[other]:
                        new_domains[other].remove(v)

            if len(new_domains[other]) == 0:
                return None  # Dead end

    return new_domains

# ------------------------------------------------
# 6) BACKTRACKING + MRV + FORWARD CHECKING
# ------------------------------------------------

def backtrack(assignment, domains):

    if len(assignment) == len(variables):
        return assignment

    course = select_unassigned_var(assignment, domains)

    for value in domains[course]:
        if is_consistent(course, assignment, value):
            new_domains = forward_check(course, value, domains, assignment)

            if new_domains is not None:
                result = backtrack({**assignment, course: value}, new_domains)
                if result is not None:
                    return result

    return None

# ------------------------------------------------
# 7) RUN THE CSP SOLVER
# ------------------------------------------------

print("Solving CSP… (MRV + Forward Checking)")
solution = backtrack({}, domains)

if solution is None:
    print("❌ No solution found!")
else:
    print("✅ Timetable successfully generated!")

# ------------------------------------------------
# 8) CONVERT SOLUTION TO DATAFRAME
# ------------------------------------------------

sol_list = []
for c, (room, day, time) in solution.items():
    teacher = courses[courses["Course"] == c]["Teacher"].values[0]
    subject = courses[courses["Course"] == c]["Subject"].values[0]
    sol_list.append([c, teacher, room, day, time, subject])

timetable_df = pd.DataFrame(sol_list, columns=["Course", "Teacher", "Room", "Day", "Time", "Subject"])
timetable_df = timetable_df.sort_values(["Day", "Time"])

print("\nGenerated Timetable:")
print(timetable_df)

# ------------------------------------------------
# 9) BEAUTIFUL HTML GUI TABLE (Color + Hover)
# ------------------------------------------------

def display_colored_gui_table(df):
    unique_subjects = df['Subject'].unique()
    colors = [
        "#ff9999", "#ffcc99", "#ffff99", "#ccff99",
        "#99ffcc", "#99ffff", "#99ccff", "#cc99ff",
        "#ff99cc", "#ff6699"
    ]

    subject_colors = {sub: colors[i % len(colors)] for i, sub in enumerate(unique_subjects)}

    html = """
    <style>
        table.timetable {
            font-family: Arial, sans-serif;
            border-collapse: collapse;
            width: 100%;
            text-align: center;
            margin-top: 15px;
        }
        table.timetable th {
            background-color: #222;
            color: white;
            padding: 10px;
        }
        table.timetable td {
            padding: 8px;
            border: 1px solid #888;
            font-size: 14px;
        }
        table.timetable tr:hover {
            background-color: #e6e6e6;
        }
    </style>
    <table class="timetable">
        <tr>
            <th>Course</th>
            <th>Teacher</th>
            <th>Room</th>
            <th>Day</th>
            <th>Time</th>
            <th>Subject</th>
        </tr>
    """

    for _, row in df.iterrows():
        color = subject_colors[row["Subject"]]
        html += f"""
        <tr>
            <td>{row['Course']}</td>
            <td>{row['Teacher']}</td>
            <td>{row['Room']}</td>
            <td>{row['Day']}</td>
            <td>{row['Time']}</td>
            <td style="background-color:{color}; font-weight:bold;">
                {row['Subject']}
            </td>
        </tr>
        """

    html += "</table>"
    display(HTML(html))

# Display GUI Timetable
print("\n📌 GUI Timetable (HTML Below):")
display_colored_gui_table(timetable_df)


Solving CSP… (MRV + Forward Checking)
✅ Timetable successfully generated!

Generated Timetable:
  Course Teacher Room  Day   Time       Subject
3    OOP   Sir C   R1  Mon    1-2   Programming
1     DB   Sir A   R1  Mon  10-11     Databases
2     DS   Sir B   R1  Mon  11-12  Data Science
4     SE   Sir D   R1  Mon    2-3  Software Eng
0     AI   Sir A   R1  Mon   9-10            AI

📌 GUI Timetable (HTML Below):


Course,Teacher,Room,Day,Time,Subject
OOP,Sir C,R1,Mon,1-2,Programming
DB,Sir A,R1,Mon,10-11,Databases
DS,Sir B,R1,Mon,11-12,Data Science
SE,Sir D,R1,Mon,2-3,Software Eng
AI,Sir A,R1,Mon,9-10,AI
